In [1]:
import datetime
from dateutil import tz
import fsspec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

### Read in the inventory 

In [2]:
df = pd.read_csv('caco-01_inventory.txt', header=None, delim_whitespace=True, parse_dates={'datetime': [0, 1]})
df.rename(columns={2:'fid',3:'filename'}, inplace=True)
del df['fid']
df

,datetime,filename
0,2019-12-11 15:45:06,1576094400.c1.snap.jpg
1,2019-12-11 15:45:07,1576094400.c2.snap.jpg
2,2019-12-11 15:45:08,1576094401.c1.timex.jpg
3,2019-12-11 15:45:08,1576094401.c2.timex.jpg
4,2019-12-11 15:45:09,1576094402.c1.var.jpg
...,...,...
89608,2020-09-25 16:04:50,1601062203.c2.bright.jpg
89609,2020-09-25 16:04:53,1601062204.c1.dark.jpg
89610,2020-09-25 16:04:57,1601062204.c2.dark.jpg
89611,2020-09-25 16:04:59,1601062205.c1.rundark.jpg


In [3]:
# find instances of timex.c1
df2 = df[df['filename'].str.contains('c1.timex')]
df2

,datetime,filename
2,2019-12-11 15:45:08,1576094401.c1.timex.jpg
14,2019-12-11 15:45:15,1576096201.c1.timex.jpg
26,2019-12-11 16:45:06,1576098001.c1.timex.jpg
38,2019-12-11 16:45:16,1576099801.c1.timex.jpg
50,2019-12-12 11:45:07,1576166401.c1.timex.jpg
...,...,...
89555,2020-09-25 13:59:41,1601055001.c1.timex.jpg
89567,2020-09-25 15:07:12,1601056801.c1.timex.jpg
89579,2020-09-25 15:08:32,1601058601.c1.timex.jpg
89591,2020-09-25 16:03:51,1601060401.c1.timex.jpg


In [4]:
# example of how to test to see if a specific record exists
len(df[df['filename'].isin(['1576094401.c2.timex.jpg'])])

1

#### Find all of the `c1.timex` images, then loop through and see if they have matching `c2` images

In [5]:
# find instances of timex.c1
df2 = df[df['filename'].str.contains('c1.timex')]
df2

,datetime,filename
2,2019-12-11 15:45:08,1576094401.c1.timex.jpg
14,2019-12-11 15:45:15,1576096201.c1.timex.jpg
26,2019-12-11 16:45:06,1576098001.c1.timex.jpg
38,2019-12-11 16:45:16,1576099801.c1.timex.jpg
50,2019-12-12 11:45:07,1576166401.c1.timex.jpg
...,...,...
89555,2020-09-25 13:59:41,1601055001.c1.timex.jpg
89567,2020-09-25 15:07:12,1601056801.c1.timex.jpg
89579,2020-09-25 15:08:32,1601058601.c1.timex.jpg
89591,2020-09-25 16:03:51,1601060401.c1.timex.jpg


In [6]:
filelist = [ ]
matches = 0
no_matches = 0
for i, row in df2.iterrows():
    fs1 = row['filename']
    # print(fs1)
    fn1 = list(fs1)
    fn2 = fn1.copy()
    fn2[12]='2'
    fs2="".join(fn2)
    # print(fs2)
    if len(df[df['filename'].isin([fs2])]) == 1:
        filelist.append(fs1)
        matches += 1
    else:
        no_matches += 1

print(matches, no_matches, matches+no_matches)    

7208 510 7718


In [7]:
cams = ['c1','c2']
pics = ['snap','timex','bright','dark','var']
for c in cams:
    for p in pics:
        s = c+'.'+p
        print(s, df['filename'].str.contains(s).sum())

c1.snap 7720
c1.timex 7718
c1.bright 7718
c1.dark 7718
c1.var 7718
c2.snap 7218
c2.timex 7217
c2.bright 7217
c2.dark 7217
c2.var 7217


In [8]:
print(7218/7720)

0.9349740932642487


In [9]:
date_time_str = '2020-01-01 00:00'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
print(date_time_obj)
utc = datetime.datetime.utcfromtimestamp(1600716600)
print(utc)

2020-01-01 00:00:00
2020-09-21 19:30:00


In [10]:
hr_list = ['06','07','08','09','10','11','12','13','14','15','16','17','18','19']
mn_list = ['00','30']
for h in hr_list:
    for m in mn_list:
        dts = ('2020-01-01 '+h+':'+m)
        dto = datetime.datetime.strptime(dts, '%Y-%m-%d %H:%M')
        print(dto.time())

06:00:00
06:30:00
07:00:00
07:30:00
08:00:00
08:30:00
09:00:00
09:30:00
10:00:00
10:30:00
11:00:00
11:30:00
12:00:00
12:30:00
13:00:00
13:30:00
14:00:00
14:30:00
15:00:00
15:30:00
16:00:00
16:30:00
17:00:00
17:30:00
18:00:00
18:30:00
19:00:00
19:30:00
